Import necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras import regularizers
from tensorflow import keras
from keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
import string
from google.colab import drive
import math
import re # module to remove non-alphanumeric chars

Load and view data

In [ ]:
# Load data
#drive.mount('/content/drive')

#df = pd.read_csv("./drive/MyDrive/Datasets/RateMyProfessor_Sample_data.csv")

#print(df.head(3))

MessageError: ignored

In [ ]:
# data loading
df=pd.read_csv('RateMyProfessor_Sample data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   professor_name            20000 non-null  object 
 1   school_name               20000 non-null  object 
 2   department_name           20000 non-null  object 
 3   local_name                20000 non-null  object 
 4   state_name                20000 non-null  object 
 5   year_since_first_review   20000 non-null  float64
 6   star_rating               20000 non-null  float64
 7   take_again                2998 non-null   object 
 8   diff_index                20000 non-null  float64
 9   tag_professor             11093 non-null  object 
 10  num_student               20000 non-null  float64
 11  post_date                 19995 non-null  object 
 12  name_onlines              20000 non-null  object 
 13  name_not_onlines          19995 non-null  object 
 14  studen

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   professor_name            20000 non-null  object 
 1   school_name               20000 non-null  object 
 2   department_name           20000 non-null  object 
 3   local_name                20000 non-null  object 
 4   state_name                20000 non-null  object 
 5   year_since_first_review   20000 non-null  float64
 6   star_rating               20000 non-null  float64
 7   take_again                2998 non-null   object 
 8   diff_index                20000 non-null  float64
 9   tag_professor             11093 non-null  object 
 10  num_student               20000 non-null  float64
 11  post_date                 19995 non-null  object 
 12  name_onlines              20000 non-null  object 
 13  name_not_onlines          19995 non-null  object 
 14  studen

In [ ]:
# Check types of the features
df.dtypes

professor_name               object
school_name                  object
department_name              object
local_name                   object
state_name                   object
year_since_first_review     float64
star_rating                 float64
take_again                   object
diff_index                  float64
tag_professor                object
num_student                 float64
post_date                    object
name_onlines                 object
name_not_onlines             object
student_star                float64
student_difficult           float64
attence                      object
for_credits                  object
would_take_agains            object
grades                       object
help_useful                 float64
help_not_useful             float64
comments                     object
word_comment                float64
gender                       object
race                         object
asian                       float64
hispanic                    

Extract data and pre-process

In [ ]:
# Extract comments and the 2 labels
df = df.iloc[:, [22, 14, 15]]
df.head(3)

,comments,student_star,student_difficult
0,"This class is hard, but its a two-in-one gen-e...",5.0,3.0
1,Definitely going to choose Prof. Looney\'s cla...,5.0,2.0
2,I overall enjoyed this class because the assig...,4.0,3.0


In [ ]:
# Find any NaN in the two labels
print(np.where(np.isnan(df['student_star'])))
print(np.where(np.isnan(df['student_difficult'])))

# Drop the observations that contain a NaN(s) as a label
df = df.dropna(subset=["student_star"])

(array([693, 694, 695, 696, 697]),)
(array([693, 694, 695, 696, 697]),)


In [ ]:
# Check if NaN were successfully removed
print(np.where(np.isnan(df['student_star'])))
print(np.where(np.isnan(df['student_difficult'])))

print("New shape", df.shape)

(array([], dtype=int64),)
(array([], dtype=int64),)
New shape (19995, 3)


In [ ]:
# Extract comments, student star rating, and student difficulty rating
x = df["comments"].astype(str).tolist()
y_star = df.iloc[:, 1].values
y_diff = df.iloc[:, 2].values

print(type(x), type(y_star), type(y_diff))
print(len(x), y_star.size, y_diff.size)

<class 'list'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
19995 19995 19995


In [ ]:
# Removes any character that is not a word or whitespace while lowercasing
cleaning_x = []
for i in x:
  i = re.sub(r'[^\w\s]', ' ', i)
  i = i.lower()
  cleaning_x.append(i)

# Remove \
cleaned_x = []
for j in cleaning_x:
  j = j.replace("\\", " ")
  cleaned_x.append(j)

# Remove stopwords if have time
# Get NLTK stopwords
nltk.download('stopwords')
cleanest_text = []
for q in cleaned_x:
  words = q.split()
  cleaned = []
  for w in words:
    if w not in set(stopwords.words('english')):
      cleaned.append(w)
  cleanest_text.append(' '.join(cleaned))

#cleanest_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(cleanest_text)

19995

In [ ]:
# Tokenization
tokens = tf.keras.preprocessing.text.Tokenizer(num_words = 20000)

# Index vocabulary, so can make sequence later
tokens.fit_on_texts(cleanest_text)

# Convert the student comments to a sequence of integers
integer_sequences = tokens.texts_to_sequences(cleanest_text)

# Take a look at one student comment in the form of an integer sequence
print(integer_sequences[69])

[105, 1206, 51, 2985, 1279, 155, 32, 16, 274, 45, 43, 138, 766, 392, 3531, 1, 1456, 24, 5255, 698, 1052, 164, 1069, 380, 1859, 2208, 1]


In [ ]:
# Padding the sequence to make them all the same lengths
# Note: Sequences must have same length to feed into NN

# Find and set max length from sequences to pad with zeroes
max_length = 0
for k in integer_sequences:
  if len(k) > max_length:
    max_length = len(k)
print("Max length:", max_length)

padded_sequences = tf.keras.utils.pad_sequences(integer_sequences,
                                                maxlen = max_length,
                                                padding = "post")
print(padded_sequences.shape)
print(padded_sequences[69])
print(integer_sequences[69])

Max length: 72
(19995, 72)
[ 105 1206   51 2985 1279  155   32   16  274   45   43  138  766  392
 3531    1 1456   24 5255  698 1052  164 1069  380 1859 2208    1    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[105, 1206, 51, 2985, 1279, 155, 32, 16, 274, 45, 43, 138, 766, 392, 3531, 1, 1456, 24, 5255, 698, 1052, 164, 1069, 380, 1859, 2208, 1]


Load in Stanford's GloVe, which is a pre-trained word embedding application

In [ ]:
# Chollet's Style
embeddings_index = {}
with open("glove.6B.100d.txt") as f:
  for line in f:
    word, coefs = line.split(maxsplit = 1)
    coefs = np.fromstring(coefs, "f" ,sep = " ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))
#embeddings_index.keys()
#embeddings_index['is']

Found 400000 word vectors.


In [ ]:
# Preparing/Creating the GloVe word-embeddings matrix
embedding_dimensions = 100
word_index = tokens.word_index

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimensions))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(16228, 100)

In [ ]:
max_length

72

In [ ]:
len(word_index)

16227

In [ ]:
# Define the Embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim = len(word_index) + 1,
                                            output_dim = embedding_dimensions,
                                            input_length = max_length,
                                            weights = [embedding_matrix],
                                            embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
                                            trainable = False,
                                            mask_zero=True)

In [ ]:
padded_sequences.shape

(19995, 72)

In [ ]:
padded_sequences[0].shape[0]

72

In [ ]:
# Use Embedding Layer to convert integer sequences to vector sequences
# skip for now, since i think it is being embedded already during training process due to the first embedding layer
#embedded_sequences = embedding_layer(padded_sequences)

In [ ]:
#embedded_sequences.shape

Splitting the data into 60% training, 20% validation, 20% testing while shuffling

In [ ]:
# Convert embedded_sequences from Tensor to np.array for splitting
# Note: Convert from np.array to Tensor after to feed to NN
x_np = padded_sequences
print(x_np.shape, type(x_np))
print(y_star.shape, y_diff.shape)

(19995, 72) <class 'numpy.ndarray'>
(19995,) (19995,)


In [ ]:
x_tv, x_test, ys_tv, ys_test, yd_tv, yd_test=train_test_split(
    x_np,y_star,y_diff,test_size=0.2, shuffle=True)
x_train,x_val,ys_train,ys_val,yd_train,yd_val=train_test_split(
    x_tv,ys_tv,yd_tv,test_size=0.25, shuffle=True)
print(x_train.shape, x_val.shape, x_test.shape)
print(ys_train.shape, ys_val.shape, ys_test.shape)
print(yd_train.shape, yd_val.shape, yd_test.shape)

(11997, 72) (3999, 72) (3999, 72)
(11997,) (3999,) (3999,)
(11997,) (3999,) (3999,)


In [ ]:
# Convert from np.array to Tensor after to feed to NN
x_train = tf.convert_to_tensor(x_train)
x_val = tf.convert_to_tensor(x_val)
x_test = tf.convert_to_tensor(x_test)
ys_train = tf.convert_to_tensor(ys_train)
ys_val = tf.convert_to_tensor(ys_val)
ys_test = tf.convert_to_tensor(ys_test)
yd_train = tf.convert_to_tensor(yd_train)
yd_val = tf.convert_to_tensor(yd_val)
yd_test = tf.convert_to_tensor(yd_test)

In [ ]:
x_train.shape

TensorShape([11997, 72])

In [ ]:
ys_train = to_categorical(ys_train, num_classes=9)
ys_val = to_categorical(ys_val, num_classes=9)
ys_test = to_categorical(ys_test, num_classes=9)
yd_train = to_categorical(yd_train, num_classes=9)
yd_val = to_categorical(yd_val, num_classes=9)
yd_test = to_categorical(yd_test, num_classes=9)

The models and layers

In [ ]:
x_train[0]

<tf.Tensor: shape=(72,), dtype=int32, numpy=
array([   13,   116,   162,   190,    99,   664,   321,  4303,    83,
        1791,   297,   126,   124,   145,    61,  1469,   358,   190,
        4338,  1939,    21,   106,  2378, 13832,    18,  1186,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)>

In [ ]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=15, batch_size=32)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 72, 100)           1622800   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               34048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 9)                 585       
                                                                 
Total params: 1,657,433
Trainable params: 34,633
Non-trainable params: 1,622,800
_________________________________________________________________
Epoch 1/15
375/375 [==============================] - 48s 105ms/step - loss: 1.4342 - accuracy: 0.4004 - va

In [ ]:
import pandas as pd
import random

# Load the dataset into a pandas dataframe
df = pd.read_csv("RateMyProfessor_Sample data.csv")

# Define the text shuffling function
def shuffle_text(text):
    words = text.split()
    random.shuffle(words)
    shuffled_text = ' '.join(words)
    return shuffled_text

# Apply the text shuffling function to each sentence in the dataset
augmented_data = []
for sentence in df['comments']:
    augmented_sentence = shuffle_text(comments)
    augmented_data.append(comments)

# Add the augmented data to the original dataframe
df_augmented = pd.DataFrame({'comments': augmented_data, 'label': df['label']})

# Concatenate the original and augmented dataframes
df_concatenated = pd.concat([df, df_augmented], ignore_index=True)

# Save the concatenated dataframe to a new csv file
df_concatenated.to_csv("my_augmented_dataset.csv", index=False)


AttributeError: ignored

In [ ]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=15, batch_size=32)


In [ ]:
model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

model.fit(x_train, yd_train, validation_data=(x_val, yd_val),
           epochs=15, batch_size=32)

In [ ]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

model = keras.Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.2)))
model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(LSTM(32, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(9, activation='softmax'))

model.summary()
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=15, batch_size=32)

In [ ]:
# This works: Will be a baseline ~ Around 50% accuracy
# Bidirectional GRU, did not yield good results at all, overfitting and continuous decrease performance per epoch
model = keras.Sequential()
model.add(embedding_layer)
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(9, activation='softmax'))
model.summary()
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=15, batch_size=32)

In [ ]:

model = keras.Sequential()
model.add(embedding_layer)

model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(9, activation='softmax'))
model.summary()
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(x_train, ys_train, validation_data=(x_val, ys_val),
           epochs=15, batch_size=32)

In [ ]:
# This works: Will be a baseline ~ Around 50% accuracy
model = keras.Sequential()
model.add(embedding_layer)
model.add(GRU(32))
model.add(Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())
model.add(Dense(9, activation='softmax'))
model.summary()
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.03) # default 0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(x_train, yd_train, validation_data=(x_val, yd_val),
           epochs=15, batch_size=32)

web scrapping


In [ ]:
import requests

# Define the URL to be scraped
url = "https://www.ratemyprofessors.com/professor/1118877"

# Send a GET request to the URL
response = requests.get(url)

# Get the HTML content of the response
html_content = response.content

# Save the HTML content to a file with .html extension
with open("ratemyproff.html", "wb") as f:
    f.write(html_content)

In [ ]:
from bs4 import BeautifulSoup
with open('ratemyproff.html','r') as html_file:
  content = html_file.read()
  soup = BeautifulSoup(content,'lxml')
  comments = soup.find_all('div', class_='Comments__StyledComments-dzzyvm-0 gRjWel')
  for course in comments:
    print(comments)


[<div class="Comments__StyledComments-dzzyvm-0 gRjWel">He teaches NOTHING during lecture, no code, no helpful lecture. Not Grading for the whole semester and he'll grade it after final project. Member of project are random. Project 1 is at almost first half semester, and Project 2,3,4 are on the last month of semester WHYY. Please AVOID at all COST and don't believe in positive review.</div>, <div class="Comments__StyledComments-dzzyvm-0 gRjWel">One of the best professors at csuf for cs department in my opinion. Just make sure to keep up with the learning material and don't miss any assignments because they build on top of each other concept wise. </div>, <div class="Comments__StyledComments-dzzyvm-0 gRjWel">He didn't grade for a long time. The greatest challenge is trying to figure out how his notes apply to the assignment.
</div>, <div class="Comments__StyledComments-dzzyvm-0 gRjWel">He does not grade until the end of the semester, until then you have no idea what your grade will be.